In [1]:
# import classification libraries
import datetime

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [3]:
# read in ecg data
measurements = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)

In [4]:
# get labels from dataset
x = measurements.iloc[:, :-1]
y = measurements.iloc[:, -1]

# change x to an array
x = np.array(x[:])

# split dataset into training and test set.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=120)

# reshape dataset so the CNN is able to read it.
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

# reshape training and test labels for input in the CNN
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [5]:
# initialize variables
batch_size = 256
num_classes = 2
epochs = 15
input_shape = (x_train.shape[1], 1)

In [6]:
# initialize model
model = keras.models.Sequential([
    keras.layers.Conv1D(128, kernel_size=3, padding='same', activation='relu', input_shape=input_shape),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(128, kernel_size=3, padding='same', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='tanh'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='tanh'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(num_classes, activation='softmax')
])
model.summary()  # print a summary of the model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 140, 128)          512       
_________________________________________________________________
batch_normalization (BatchNo (None, 140, 128)          512       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 70, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 70, 128)           49280     
_________________________________________________________________
batch_normalization_1 (Batch (None, 70, 128)           512       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 35, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 4480)              0

In [8]:
# compile and train model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])

# creating a logs directory, although im not sure how this acts on co-lab
# since i ran this code on my own computer.
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_70_30"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# fit model, don't forget the callback.
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[tensorboard_callback])

# get predictions and reshape them back to a 1D array for metrics
y_pred = model.predict(x_test)
y_pred = np.nanargmax(y_pred, axis=1)
y_test = np.nanargmax(y_test, axis=1)

print("F1 Score:  ", f1_score(y_test, y_pred, average="macro"))
print("Precision: ", precision_score(y_test, y_pred, average="macro"))
print("Recall:    ", recall_score(y_test, y_pred, average="macro"))
print("Accuracy:  ", np.nanmean((y_test == y_pred) * 1.0))

Epoch 1/15
14/14 [==============================] - 5s 273ms/step - loss: 0.0819 - accuracy: 0.9888 - precision: 0.9888 - recall: 0.9888
Epoch 2/15
14/14 [==============================] - 4s 261ms/step - loss: 0.0566 - accuracy: 0.9905 - precision: 0.9905 - recall: 0.9905
Epoch 3/15
14/14 [==============================] - 4s 263ms/step - loss: 0.0480 - accuracy: 0.9919 - precision: 0.9919 - recall: 0.9919
Epoch 4/15
14/14 [==============================] - 4s 265ms/step - loss: 0.0412 - accuracy: 0.9920 - precision: 0.9920 - recall: 0.9920
Epoch 5/15
14/14 [==============================] - 4s 268ms/step - loss: 0.0359 - accuracy: 0.9951 - precision: 0.9951 - recall: 0.9951
Epoch 6/15
14/14 [==============================] - 4s 271ms/step - loss: 0.0292 - accuracy: 0.9950 - precision: 0.9950 - recall: 0.9950
Epoch 7/15
14/14 [==============================] - 4s 272ms/step - loss: 0.0274 - accuracy: 0.9952 - precision: 0.9952 - recall: 0.9952
Epoch 8/15
14/14 [=======================